<a href="https://colab.research.google.com/github/Nagib005/Inverse_Design/blob/main/Forward_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adadelta

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Specify the path to your CSV file in Drive
file_path = '/content/drive/MyDrive/Circular_inverse/result_V.csv'  # <-- Change this to your actual path

# Step 3: Load the data from CSV file (results of FDTD solver)
result = pd.read_csv(file_path, header=None)
result = result.to_numpy()

x = result[:, 0:6]
y = result[:, 6:8]

# Allocation of 70% of the total data to the training data
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.30, shuffle=True)
# Allocation of 50% of the remaining data to the validation data and 50% to the test data (15% validation and 15% test of total)
x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size=0.50, shuffle=True)

# Feature Scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)
x_test = sc.transform(x_test)

# Free up RAM
del result

# Define the Layers of the deep neural network (DNN)
Model = Sequential()
Model.add(Dense(60, input_dim=6))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
Model.add(Dense(60))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
Model.add(Dense(60))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
Model.add(Dense(60))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
Model.add(Dense(60))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
Model.add(Dense(60))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dense(2))
Model.summary()

# EarlyStopping callback
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=2)

# Compile model
Model.compile(loss='mean_squared_logarithmic_error',
              optimizer=Adadelta(learning_rate=0.1))

# Train model
history = Model.fit(x_train, y_train, epochs=5000,
                    batch_size=80, validation_data=(x_val, y_val), callbacks=[es])

# Plot loss graph
plt.plot(history.history['val_loss'], linewidth=1)
plt.plot(history.history['loss'], linewidth=2, linestyle='--')
plt.title('The loss of training model', fontname='Times New Roman', fontsize=18, loc='center')
plt.xlabel('epochs', fontname='Times New Roman', fontsize=18)
plt.ylabel('loss', fontname='Times New Roman', fontsize=18)
plt.xticks(fontfamily='Times New Roman', fontsize=14)
plt.yticks(fontfamily='Times New Roman', fontsize=14)
plt.legend(['Validation Loss', 'Training Loss'])
plt.show()

# Loss values of train and validation data
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Test model
predictions = Model.predict(x_test)
Loss = Model.evaluate(x_test, y_test)
print('Test loss:', Loss)

# Save loss values to pickle and CSV files
with open('history_Forward_model.pkl', 'wb') as f:
    pickle.dump(history.history, f)

fieldnames = ['Epoch', 'Training Loss', 'Validation Loss']
with open('loss_Forward_model.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for epoch, train_loss_value, val_loss_value in zip(range(1, len(train_loss) + 1), train_loss, val_loss):
        writer.writerow({'Epoch': epoch, 'Training Loss': train_loss_value, 'Validation Loss': val_loss_value})

# Save model architecture and weights
model_json = Model.to_json()
with open("T-shaped_switch_Nozhat_model.json", "w") as json_file:
    json_file.write(model_json)

Model.save_weights("T-shaped_switch_Nozhat_model_weights.h5")


Mounted at /content/drive


In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adadelta
import matplotlib.pyplot as plt
import pickle
import csv

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Path to your CSV file in Drive — update this
file_path = '/content/drive/MyDrive/Circular_inverse/result_V.csv'

# Step 3: Define chunk size and dtype for memory efficiency
chunksize = 30000
dtypes = {i: 'float32' for i in range(8)}  # Assuming 8 columns total

# Step 4: Read CSV in chunks and collect data
x_chunks = []
y_chunks = []

for chunk in pd.read_csv(file_path, header=None, chunksize=chunksize, dtype=dtypes):
    x_chunks.append(chunk.iloc[:, 0:6].values)
    y_chunks.append(chunk.iloc[:, 6:8].values)

# Step 5: Stack chunks vertically into full arrays
x = np.vstack(x_chunks)
y = np.vstack(y_chunks)

# Step 6: Split data into training, validation, and test sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.30, shuffle=True)
x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size=0.50, shuffle=True)

# Step 7: Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)
x_test = sc.transform(x_test)

# Clear memory of raw chunks
del x_chunks, y_chunks, x, y

# Step 8: Define the model architecture
Model = Sequential()
Model.add(Dense(60, input_dim=6))
Model.add(LeakyReLU(alpha=0.2))
Model.add(Dropout(0.1))
for _ in range(5):
    Model.add(Dense(60))
    Model.add(LeakyReLU(alpha=0.2))
    Model.add(Dropout(0.1))
Model.add(Dense(2))
Model.summary()

# Step 9: Compile the model
Model.compile(loss='mean_squared_logarithmic_error', optimizer=Adadelta(learning_rate=0.1))

# Step 10: Early stopping callback
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=2)

# Step 11: Train the model
history = Model.fit(x_train, y_train, epochs=5000, batch_size=80,
                    validation_data=(x_val, y_val), callbacks=[es])

# Step 12: Plot training and validation loss
plt.plot(history.history['val_loss'], linewidth=1)
plt.plot(history.history['loss'], linewidth=2, linestyle='--')
plt.title('Training Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Validation Loss', 'Training Loss'])
plt.show()

# Step 13: Evaluate on test set
loss = Model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')

# Step 14: Save training history and model
with open('history_Forward_model.pkl', 'wb') as f:
    pickle.dump(history.history, f)

fieldnames = ['Epoch', 'Training Loss', 'Validation Loss']
with open('loss_Forward_model.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for epoch, train_loss_value, val_loss_value in zip(range(1, len(history.history['loss']) + 1),
                                                      history.history['loss'], history.history['val_loss']):
        writer.writerow({'Epoch': epoch, 'Training Loss': train_loss_value, 'Validation Loss': val_loss_value})

model_json = Model.to_json()
with open("T-shaped_switch_Nozhat_model.json", "w") as json_file:
    json_file.write(model_json)
Model.save_weights("T-shaped_switch_Nozhat_model_weights.h5")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout
from keras.optimizers import Adadelta

# Mount Google Drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Circular_inverse/result_V.csv'  # <-- Update this!

chunksize = 30000
dtypes = {i: 'float32' for i in range(8)}  # assuming 8 columns total

# Step 1: Initialize scaler and fit incrementally (partial fit)
scaler = StandardScaler()

print("Fitting scaler incrementally on chunks...")
for chunk in pd.read_csv(file_path, header=None, chunksize=chunksize, dtype=dtypes):
    x_chunk = chunk.iloc[:, 0:6].values
    scaler.partial_fit(x_chunk)

# Step 2: Define the model architecture
model = Sequential()
model.add(Dense(60, input_dim=6))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.1))
for _ in range(5):
    model.add(Dense(60))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.1))
model.add(Dense(2))

model.compile(loss='mean_squared_logarithmic_error', optimizer=Adadelta(learning_rate=0.1))

# Step 3: Incremental training loop
epochs = 10  # Number of full passes over data

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for chunk in pd.read_csv(file_path, header=None, chunksize=chunksize, dtype=dtypes):
        x_chunk = chunk.iloc[:, 0:6].values
        y_chunk = chunk.iloc[:, 6:8].values
        x_chunk = scaler.transform(x_chunk)
        # Train on current chunk
        loss = model.train_on_batch(x_chunk, y_chunk)
    print(f"Last chunk training loss: {loss:.5f}")

# Step 4 (Optional): Save your trained model
model.save_weights('/content/drive/MyDrive/Circular_inverse/T-shaped_switch_Nozhat_model_weights.h5')
model_json = model.to_json()
with open("T-shaped_switch_Nozhat_model.json", "w") as json_file:
    json_file.write(model_json)

print("Training complete and model saved.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fitting scaler incrementally on chunks...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Epoch 1/10
Last chunk training loss: 0.00571

Epoch 2/10
Last chunk training loss: 0.00459

Epoch 3/10
Last chunk training loss: 0.00418

Epoch 4/10
Last chunk training loss: 0.00396

Epoch 5/10
Last chunk training loss: 0.00382

Epoch 6/10
Last chunk training loss: 0.00372

Epoch 7/10
Last chunk training loss: 0.00365

Epoch 8/10
Last chunk training loss: 0.00360

Epoch 9/10
Last chunk training loss: 0.00356

Epoch 10/10
Last chunk training loss: 0.00352


ValueError: The filename must end in `.weights.h5`. Received: filepath=T-shaped_switch_Nozhat_model_weights.h5

In [ ]:
# Step 4 (Optional): Save your trained model
model.save_weights('/content/drive/MyDrive/Circular_inverse/T-shaped_switch_Nozhat_model.weights.h5')
model_json = model.to_json()
with open("T-shaped_switch_Nozhat_model.json", "w") as json_file:
    json_file.write(model_json)

print("Training complete and model saved.")

Training complete and model saved.
